In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci

I build this scrit to see wether I can reproduce the electron electron interaction energy form the 2RDM, this serves as test to see wether I understand the data struvctures and theorie accurately to move on to other ideas.

The following function caculates the electron electron interactio  energy from the Fock basis representation of the 2RDM $\gamma_2$ in the follwong way:
$$E_{ee} = \sum_p \sum_q \sum_r \sum_s \gamma_{2_{pqrs}} \langle p q | r s \rangle$$

In [2]:
def ee_from_gamma_2(eri,dm2):
    E = 0 
    dim = eri.shape[0]
    for p in range(0,dim):
        for q in range(0,dim):
            for r in range(0,dim):
                for s in range(0,dim):
                    E+=eri[p,q,r,s]*dm2[p,q,r,s]
    return E

In [3]:
mol = gto.Mole()
mol.atom = """
    Li    0.    0.    0.
"""
mol.basis = "cc-pvdz"
#mol.basis = "6-31g"
mol.spin=1
mol.build()
print(mol.nelec)

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri_ao = mol.intor('int2e')

(2, 1)


In [4]:
# Run Hartree-Fock.
mf = scf.UHF(mol)
mf.kernel()

converged SCF energy = -7.43242052759577  <S^2> = 0.75000054  2S+1 = 2.0000005


np.float64(-7.432420527595772)

In [5]:
# Harvest HF quantities

h1 = mf.get_hcore()
M = h1.shape[0]
C_a = mf.mo_coeff[0,:,:]
C_b = mf.mo_coeff[1,:,:]

h1_a = C_a.T@h1@C_a
h1_b = C_b.T@h1@C_b

eri_h2_aa = eri_ao.copy()
eri_h2_bb = eri_ao.copy()
eri_h2_ab = eri_ao.copy()


for i in range(4):
  eri_h2_aa = np.tensordot(eri_h2_aa, C_a, axes=1).transpose(3, 0, 1, 2)
  eri_h2_bb = np.tensordot(eri_h2_bb, C_b, axes=1).transpose(3, 0, 1, 2)

eri_h2_ab = np.tensordot(eri_h2_ab, C_a, axes=1).transpose(3, 0, 1, 2)
eri_h2_ab = np.tensordot(eri_h2_ab, C_a, axes=1).transpose(3, 0, 1, 2)
eri_h2_ab = np.tensordot(eri_h2_ab, C_b, axes=1).transpose(3, 0, 1, 2)
eri_h2_ab = np.tensordot(eri_h2_ab, C_b, axes=1).transpose(3, 0, 1, 2)

In [6]:
h1_mo = (h1_a, h1_b)
h2_mo = (eri_h2_aa, eri_h2_ab, eri_h2_bb)

In [7]:
# Run FCI  
cisolver = fci.direct_uhf.FCI()
cisolver.max_cycle = 100
cisolver.conv_tol = 1e-8
e, ci = cisolver.kernel(h1_mo, h2_mo, h1.shape[0], mol.nelec, ecore=mf.energy_nuc())


In [8]:
# FCIgamma_a, FCIgamma_b = fci.direct_uhf.make_rdm1s(ci, h1.shape[0] , mol.nelec)

dm1, dm2 = cisolver.make_rdm12s(ci, mf.mo_coeff.shape[1], mol.nelec)

In [9]:
print(len(dm2), dm2[0].shape, dm2[1].shape, dm2[2].shape)

3 (14, 14, 14, 14) (14, 14, 14, 14) (14, 14, 14, 14)


In [10]:
print("Compare the Resulting Energies, should always be same.")

print(e-np.trace(dm1[0]@h1_mo[0])-np.trace(dm1[1]@h1_mo[1]))
print(1/2 * np.einsum('pqrs,pqrs', h2_mo[0], dm2[0])+np.einsum('pqrs,pqrs', h2_mo[1], dm2[1])+1/2 * np.einsum('pqrs,pqrs', h2_mo[2], dm2[2]))
# print(np.einsum('pqrs,pqrs', h2_mo[1], dm2[1]))
# print(1/2 * np.einsum('pqrs,pqrs', h2_mo[2], dm2[2]))

Compare the Resulting Energies, should always be same.
2.1472873217433346
2.1472873217433297


In [11]:
print("aa:", 1/2 * np.einsum('pqrs,pqrs', h2_mo[0], dm2[0]))
print("ab:",np.einsum('pqrs,pqrs', h2_mo[1], dm2[1]))
print("bb:",1/2 * np.einsum('pqrs,pqrs', h2_mo[2], dm2[2]))


aa: 0.37058703433465773
ab: 1.776700287408672
bb: 0.0
